In [1]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
df2006 = spark.read.format("csv") \
.option("header", "true") \
.option("nullValue","NA") \
.option("inferSchema", "true").load("2006.csv")
df2007 = spark.read.format("csv") \
.option("header", "true") \
.option("nullValue","NA") \
.option("inferSchema", "true").load("2007.csv")
df2008 = spark.read.format("csv") \
.option("header", "true") \
.option("nullValue","NA") \
.option("inferSchema", "true").load("2008.csv")

In [ ]:
print("Partitions")
df2006.rdd.getNumPartitions()
df2007.rdd.getNumPartitions()
df2008.rdd.getNumPartitions()

In [ ]:
print("Elements")
df2006.count()
df2007.count()
df2008.count()

In [ ]:
print("Fusionar")
df1 = df2006.union(df2007).union(df2008)
df1.count()
df1.rdd.getNumPartitions()

In [ ]:
df2 = df1.select("Year", "Month", "Origin", "Dest", "ArrDelay", "DepDelay")
df3 = df2.na.drop()

In [ ]:
print("retard total")
from pyspark.sql.functions import expr
df4 = df3.withColumn("SumDelay", expr("ArrDelay + DepDelay"))
df4.cache()

In [ ]:
"""
Quin és el retard mig, pels anys 2006, 2007 i 2008, per cada aeroport d’origen? Això és fa amb
la funció groupBy, que ens permet agrupar les dades segons el valor d’una determinada columna.
Hem de fer servir la funció agg per indicar quina operació volem fer sobre els valors de les columnes
agrupades.
Amb la següent instrucció es crea un DataFrame amb dues columnes: la primera columna és
l’aeroport d’origen i la segona columna contindrà el valor mig del retard total.
"""
from pyspark.sql.functions import avg
df5 = df4.groupBy("Origin").agg(avg("SumDelay"))
df5.show()
df5.count()
df6 = df5.withColumnRenamed("avg(SumDelay)", "Average Delay")
df6.show()

In [ ]:
#Exercici
#Ordeneu els resultats per retard mig obtingut, de menor a major, i mostreu els 20 primers resultats
#per pantalla.

In [ ]:
#Exercici
"""
Es proposa calcular el valor mig del retard només per a la ciutat de Nova York, agrupat per mes.
Hi ha un mes en que el retard és més gran que l’altre? Proveu-ho! Quina conclusió traieu? Quins
són els mesos en què hi ha més retard?
Podem agrupar per múltiples criteris. En aquest cas agrupem per any i origen
df5 = df4.groupBy("Year", "Origin").agg(avg("SumDelay"))
df5.show()
"""

In [ ]:
#Exercici
"""
Es proposa calcular el valor mig del retard només per a la ciutat de Nova York, agrupat per mes.
Hi ha un mes en que el retard és més gran que l’altre? Proveu-ho! Quina conclusió traieu? Quins
són els mesos en què hi ha més retard?
Anem a comptar el nombre de vols que hi ha entre els aeroports. Spark ens ofereix les funcions
necessàries per fer-ho:
>‌>‌> from pyspark.sql.functions import lit, count
>‌>‌> df5 = df4.groupBy("Origin", "Dest").agg(count(lit(1)))
La funció count(lit(1)) permet fer una suma del nombre d’elements en comptes de fer-ho sobre
els valors d’una determinada columna del DataFrame.
"""

In [ ]:
#Exercici
#Quin són els vols més habituals? I entre quins aeroports es realitzen? Nova York? Boston? Los
#Angeles?

In [ ]:
#User functions
def retard(temps):
    if (temps > 0):
        return 1
    else:
        return 0

In [ ]:
from pyspark.sql.types import IntegerType
spark.udf.register("retard_spark", retard, IntegerType())
df5 = df4.withColumn("Retard", expr("retard_spark(SumDelay)"))
df5.show()
Ara podem agrupar i veure el percentatge de retard dels avions
df5.groupBy("Origin").agg(avg("Retard")).sort("avg(Retard)").show()

In [ ]:
#Exercici
"""Es proposa calcular ara el percentatge mig dels vols amb retard per a la ciutat de Nova York, agrupat
per mes. Hi ha un mes en que el retard és més gran que l’altre? Proveu-ho! Quina conclusió traieu?
Els resultats són congruents amb el que heu obtingut abans?
En aquest exemple hem vist com podem aplicar funcions d’usuari pròpies que s’apliquen als
registres individuals del DataFrame. Spark també ofereix la possibilitat de definir funcions d’usuari
a aplicar a l’hora d’agrupar dades. Això ens permet aplicar altres funcions de les que ja estan
definides al Spark (max, min, avg, ...). Podreu trobar la forma de fer-ho a la documentació de Spark
(tècnicament s’anomenen User Defined Aggregation Functions).
"""
